# Compute Energies of Displacements Along Coordinate Systems
Displace each atom along the axes, in the same way that we would when computing finite differences. The code is similar to what is used in [ASE's vibration analysis](https://databases.fysik.dtu.dk/ase/ase/vibrations/vibrations.html) with a few differences:

- We perturb every pair of coordinates to compute numerical second derivatives. ASE compute the first derivatives of force to access the Hessian.
- Optionally, perturb more than pair at a time.

In [1]:
from jitterbug.utils import make_calculator
from itertools import permutations
from ase.io import write, read
from ase.db import connect
from ase import Atoms
from pathlib import Path
from tqdm import tqdm 
import numpy as np
import os

Configuration

In [2]:
starting_geometry = '../data/exact/caffeine_pm7_None.xyz'
threads = min(os.cpu_count(), 12)
step_size: float = 0.005 # Lambda parameter for an expontential distribution for the Perturbation amount
perturbs_per_evaluation: int = 2  # Number of perturbations to perform at once

Derived

In [3]:
run_name = Path(starting_geometry).name[:-4]
name, method, basis = run_name.split("_")

## Load in the Relaxed Structure
We generated a relaxed structure in the previous notebook

In [4]:
atoms = read(starting_geometry)
atoms

Atoms(symbols='O2N4C8H10', pbc=False, forces=..., calculator=SinglePointCalculator(...))

## Assemble the List of Perturbations
We are going to set up all of the perturbations needed for numerical partial second derivatives, which include [perturbing only one coordinate for the diagonal terms and every pair of coordinates for the off-diagonal](https://en.wikipedia.org/wiki/Finite_difference#Multivariate_finite_differences).

In [5]:
n_coords = len(atoms) * 3
perturbations = [
    (d * i,) for d in [-1, 1] for i in range(1, n_coords + 1)  # Start from 1 so we can encode direction with the sign (-0 == 0)
]
print(f'Collected {len(perturbations)} diagonal terms')
perturbations.extend(
    (d * i, d * j) for d in [-1, 1] for i, j in permutations(range(1, n_coords + 1), 2)  # Assumes we re-use the data from diagonal terms for the off-diagonal
)
print(f'Collected {len(perturbations)} with off-diagonal terms')

Collected 144 diagonal terms
Collected 10368 with off-diagonal terms


Combine several if desired

In [6]:
def combine_peturbations(singles: list[tuple[int, ...]], num_to_combine: int, seed: int = 1) -> list[tuple[int, ...]]:
    """Combine multiple perturbations into a single task
    
    Does them in a repeatable order and ensures that we do not 
    combine perturbations that act on the same atoms
    
    Args:
        singles: List of all perturbations to combine. 
            Each entry contains a list of coordinates to perturb (1-indexed)
            where the sign dictaes whether it is a positive or negative direction.
        num_to_combine: Number of perturbations to combine into one task.
            Some may be smaller than this number
        seed: Random seed
    Returns:
        List of combined purtbations
    """
    
    # Start by shuffling
    rng = np.random.default_rng(seed)
    shuffled = singles.copy()
    rng.shuffle(shuffled)
    
    # Combine them
    output = []
    while len(shuffled) > 0:
        # Add to the new task until we reach the desired number
        new_task = ()  # New perturbation vector
        num_combined = 0  # Number of perturbations which were combined
        new_inds = set()  # Coordinates which are perturbed in this new vector
        
        while num_combined < num_to_combine:  # Loop until we've merged enough
            for i in range(len(shuffled)):
                # Check if this new one contains only new atoms
                if all(abs(d) not in new_inds for d in shuffled[i]):
                    # Add it to the new task
                    to_add = shuffled.pop(i)
                    num_combined += 1
                    new_inds.update(abs(d) for d in to_add)
                    new_task = new_task + to_add
                    break
            else: 
                break  # If we fail to find a perturbation with new coordinates, stop looking
        
        output.append(new_task)
        
    return output

In [7]:
if perturbs_per_evaluation > 1:
    perturbations = combine_peturbations(perturbations, perturbs_per_evaluation)
    print(f'Combined {perturbs_per_evaluation} tasks into 1, reducing to {len(perturbations)}')

Combined 2 tasks into 1, reducing to 5184


Make sure the same coordinate does not appear in the same task twice

In [ ]:
for task in perturbations:
    inds = [abs(i) for i in task]
    assert len(inds) == len(set(inds))

## Run the Perturbations

Prepare the output directory

In [8]:
out_dir = Path('data') / 'along-axes'
out_dir.mkdir(exist_ok=True, parents=True)

In [9]:
db_path = out_dir / f'{run_name}_d={step_size:.2e}-N={perturbs_per_evaluation}.db'
print(f'Writing to {db_path}')

Writing to data/along-axes/caffeine_pm7_None_d=5.00e-03-N=2.db


Add the relaxed geometry if needed

In [10]:
if not db_path.is_file():
    with connect(db_path) as db:
        db.write(atoms, name='initial')

Make the calculator

In [11]:
calc = make_calculator(method, basis, num_threads=threads)

Generate the energies

In [12]:
with connect(db_path) as db:
    num_done = len(db) - 1
print(f'We have finished {num_done} perturbations already. {len(perturbations) - num_done} left to do.')

We have finished 0 perturbations already. 5184 left to do.


In [ ]:
iterator = tqdm(perturbations[num_done:])
for perturb in iterator:
    # Create the perturbation vector
    disp = np.zeros((n_coords,))
    for d in perturb:
        disp[abs(d) - 1] = (1 if abs(d) > 0 else -1) * step_size
    disp = disp.reshape((-1, 3))
    
    # Make the new atoms
    new_atoms = atoms.copy()
    new_atoms.positions += disp
    
    # Make the name for the computation
    name = "d" + "".join(f'{"+" if d > 0 else "-"}{abs(d)-1}' for d in perturb)
    iterator.set_description(name)
    
    # Compute the energy and store in the db
    new_atoms.calc = calc
    new_atoms.get_potential_energy()
    with connect(db_path) as db:
        db.write(new_atoms, name=name)

d-7-26+29+17: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 5178/5184 [11:50<00:01,  3.17it/s]